In [ ]:
%%writefile ativos.csv
Equipamento,Data_Aquisicao,Preco
Notebook,2023-02-10,100.37
Notebook,2023-04-12,100.37
Monitor,2022-10-20,98.66
Monitor,2023-03-12,98.66
Teclado,2024-03-02,25.90
Teclado,2023-12-12,25.90
Mouse,2022-07-06,18.75
Mouse,2023-12-12,18.75
Webcam,2023-01-15,55.40
Webcam,2024-06-10,55.40
Nobreak,2022-11-08,120.00
Nobreak,2024-02-18,120.00
Roteador,2022-03-05,95.80
Scanner,2023-04-22,210.50
Scanner,2024-01-30,210.50
Impressora,2023-09-10,399.90
Impressora,2024-04-01,399.90
Headset,2024-03-02,79.99
HD Externo,2024-10-20,150.00
HD Externo,2023-12-12,150.00
Tablet,2023-06-14,230.00
Tablet,2024-02-09,230.00
Tablet,2023-12-12,230.00
Servidor,2025-07-30,1200.00
Servidor,2023-05-12,1200.00
Servidor,2024-03-02,1200.00
Servidor,2022-08-18,1200.00
Notebook,2024-05-21,100.37
Notebook,2022-03-22,100.37
Nobreak,2023-07-19,120.00
Impressora,2022-12-02,399.90
Headset,2023-08-16,79.99
Monitor,2024-01-11,98.66
Monitor,2022-05-09,98.66
Monitor,2023-11-30,98.66
Projetor,2023-03-18,850.00
Projetor,2024-07-02,850.00
Switch,2022-06-25,180.00
Switch,2023-10-14,180.00
Firewall,2024-04-08,650.00
Firewall,2023-01-20,650.00
Access Point,2022-09-17,210.00
Access Point,2024-02-27,210.00
Leitor de Codigo de Barras,2023-05-05,145.90
Leitor de Codigo de Barras,2024-06-19,145.90
Impressora Termica,2023-07-12,520.00
Impressora Termica,2024-03-03,520.00
Estabilizador,2022-11-29,89.90
Estabilizador,2023-08-07,89.90
Microfone,2024-01-25,135.00
Microfone,2023-04-04,135.00
Camera IP,2023-09-15,310.00
Camera IP,2024-05-11,310.00

In [4]:
import pandas as pd
import plotly.graph_objects as go
import random

In [ ]:
# leitura do csv e conversão para DataFrame
df = pd.read_csv('/content/ativos.csv',sep=',')

# transformando datas em DateTime para realização de expressões
df['Data_Aquisicao'] = pd.to_datetime(df['Data_Aquisicao'],yearfirst=True)

# criando a coluna 'Depreciação' única para cada equipamento
uniq_equip = df['Equipamento'].unique().tolist()
depre = [random.uniform(0.00,0.1) for _ in range(len(uniq_equip))]
map = dict(zip(uniq_equip,depre))
df['Depreciação'] = df['Equipamento'].map(map)

# criando a coluna 'Valor Atual' dependendo da idade e da depreciação do equipamento
hoje = pd.Timestamp.now()
df['Idade'] = (hoje - df['Data_Aquisicao']).dt.days / 365.25

df['Valor Atual'] = df['Preco'] * (1 - (df['Depreciação'] * df['Idade']))
df = df.drop(columns=['Idade'])

# criando o menu interativo para visualização dos dados
print('<DEPRECIAÇÃO DE ATIVOS>')
print('SELECIONE O ATIVO PARA VISUALIZAÇÃO:')

for i in uniq_equip:
  print(f'{uniq_equip.index(i)} - {i}')

opcao = input()
opcao = int(opcao)

if(opcao < 0 or opcao > len(uniq_equip)):
  print('ERRO: opção selecionada inválida.')

else:
  opcao_valor = uniq_equip[opcao]
  df_filter = df[df['Equipamento'] == opcao_valor]
  df_filter = df_filter[['Equipamento','Preco','Valor Atual']]

# criando a visualização dos dados selecionados
  fig = go.Figure(data=[
    go.Bar(name='Valor de Compra', x=df_filter['Equipamento'], y=df_filter['Preco'], marker_color='#3e6bb5'),
    go.Bar(name='Valor Atual (Depreciado)', x=df_filter['Equipamento'], y=df_filter['Valor Atual'], marker_color='#b03749')])

  fig.update_layout(
    title='Comparativo de Patrimônio: Valor Inicial vs. Atual',
    xaxis_title='Identificação do Equipamento',
    yaxis_title='Valor em R$',
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig.show()